In [1]:
import os
import pprint

import keras
import numpy as np
import sklearn.metrics
import tensorflow as tf

from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import average, AveragePooling2D, concatenate, Conv2D, Conv3D, Dense, Flatten, Input, Reshape
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold

PATCH_HEIGHT = 28
PATCH_WIDTH = 28

data_dir = 'data'
if not os.path.exists('checkpoints'):
    os.mkdir('checkpoints')
if not os.path.exists('logs'):
    os.mkdir('logs')

pp = pprint.PrettyPrinter(indent=4)

Using TensorFlow backend.


In [2]:
ct_train = np.load(os.path.join(data_dir, 'ct_train.npy'))
pet_train = np.load(os.path.join(data_dir, 'pet_train.npy'))
y_train = np.load(os.path.join(data_dir, 'y_train.npy'))
y_train_targets = np.argmax(y_train, axis=1)

ct_test = np.load(os.path.join(data_dir, 'ct_test.npy'))
pet_test = np.load(os.path.join(data_dir, 'pet_test.npy'))
y_test = np.load(os.path.join(data_dir, 'y_test.npy'))

def get_train(mode=None, indices=None):
    if mode == 'ct':
        return ct_train[indices] if indices is not None else ct_train
    elif mode == 'pet':
        return pet_train[indices] if indices is not None else pet_train
    else:
        return [ct_train[indices], pet_train[indices]] if indices is not None else [ct_train, pet_train]

def get_test(mode=None):
    if mode == 'ct':
        return ct_test
    elif mode == 'pet':
        return pet_test
    else:
        return [ct_test, pet_test]

In [3]:
def confusion_matrix(y_true, y_pred):
    y_true_targets = np.argmax(y_true, axis=1)
    y_pred_targets = np.argmax(y_pred, axis=1)
    return sklearn.metrics.confusion_matrix(y_true_targets, y_pred_targets)

def accuracy(y_true, y_pred):
    y_true_targets = np.argmax(y_true, axis=1)
    y_pred_targets = np.argmax(y_pred, axis=1)
    return sklearn.metrics.accuracy_score(y_true_targets, y_pred_targets)

def f1(y_true, y_pred):
    c_matrix = confusion_matrix(y_true, y_pred)
    tp = c_matrix[1][1]
    fp = c_matrix[0][1]
    fn = c_matrix[1][0]
    return 2 * tp / (2 * tp + fn + fp)

In [4]:
def train_model(model_fn, name, batch_size=32, epochs=8, patience=2, mode=None, save=False, n_splits=5, val=True):
    print('Train...')

    best_model_path = os.path.join('checkpoints', f'best_model_{name}.h5')
    log_dir = os.path.join('logs', f'{name}')

    if not os.path.exists(log_dir):
        os.mkdir(log_dir)

    callbacks = []
    
    if val:
        callbacks.append(EarlyStopping(monitor='val_acc', patience=patience))
    
    if save:
        callbacks.append(ModelCheckpoint(best_model_path, monitor='val_acc', save_best_only=True, save_weights_only=True))
        callbacks.append(TensorBoard(log_dir=log_dir, histogram_freq=1, batch_size=batch_size, write_graph=False, write_grads=True, write_images=True))
    
    if n_splits > 1:
        cv = StratifiedKFold(n_splits=n_splits)
        split_num = 1
        preds = []
        for train, test in cv.split(ct_train, y_train_targets):
            print(f'Fold {split_num}/{n_splits}')
            model = model_fn()
            model.fit(get_train(mode, indices=train),
                      y_train[train],
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_data=(get_train(mode, indices=test), y_train[test]),
                      verbose=1,
                      shuffle=True,
                      callbacks=callbacks)
            y_preds = model.predict(get_test(mode))
            preds.append(y_preds)
            split_num += 1

        preds = np.argmax(np.mean(preds, axis=0), axis=1)
        with tf.Session() as sess:
            preds = sess.run(tf.one_hot(preds, 2))
    else:
        model = model_fn()
        model.fit(get_train(mode),
                  y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_split=0.1 if val else 0.0,
                  verbose=1,
                  shuffle=True,
                  callbacks=callbacks)
        preds = model.predict(get_test(mode))
    
    acc_score = accuracy(y_test, preds)
    f1_score = f1(y_test, preds)
    print(f'F1: {f1_score}')
    print(f'Acc: {acc_score}')
    print('\n\n')
    return f1_score, acc_score
    

def train_n_sessions(model_fn, name, n, mode=None, **kwargs):
    f1s = []
    accs = []
    
    for i in range(n):
        print(f'Round {i + 1} out of {n}')
        print('-' * 101)
        f1, acc = train_model(model_fn, name, mode=mode, **kwargs)
        f1s.append(f1)
        accs.append(acc)
    
    return f1s, accs

# Type 1: Feature-Level Fusion

In [5]:
def get_type_1_model(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))

    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    x = Conv3D(16, (2, 2, 2), activation='relu')(x)
    x = Reshape((27, 27, 16))(x)
    x = Conv2D(36, (2, 2), activation='relu')(x)
    x = Conv2D(64, (2, 2), activation='relu')(x)
    x = Conv2D(144, (2, 2), activation='relu')(x)
    x = AveragePooling2D((23, 23))(x)
    x = Flatten()(x)
    x = Dense(864, activation='relu')(x)
    x = Dense(288, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=[ct_input, pet_input], outputs=output)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()

    return model

In [6]:
f1s, accs = train_n_sessions(get_type_1_model, 'type_I', 10, epochs=5, n_splits=1, val=False)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 27s - loss: 0.1191 - acc: 0.9581    
Epoch 2/5
41338/41338 [==============================] - 23s - loss: 0.0850 - acc: 0.9687    
Epoch 3/5
41338/41338 [==============================] - 23s - loss: 0.0652 - acc: 0.9781    
Epoch 4/5
41338/41338 [==============================] - 23s - loss: 0.0521 - acc: 0.9837    
Epoch 5/5
41338/41338 [==============================] - 23s - loss: 0.0465 - acc: 0.9858    
F1: 0.8683329038907498
Acc: 0.8786799620132953



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 23s - loss: 0.1133 - acc: 0.9613    
Epoch 2/5
41338/41338 [==============================] - 23s - loss: 0.0862 - acc: 0.9693    
Epoch 3/5
41338/41338 [===========

In [7]:
pp.pprint(f1s)
pp.pprint(accs)

[   0.8683329038907498,
    0.87653061224489792,
    0.84892086330935257,
    0.93224411340701585,
    0.89907799651133813,
    0.8646423057128152,
    0.91441111923920992,
    0.79249509941192942,
    0.88888888888888884,
    0.9248916706788638]
[   0.87867996201329535,
    0.88509021842355173,
    0.86039886039886038,
    0.9330484330484331,
    0.90384615384615385,
    0.87511870845204176,
    0.91666666666666663,
    0.82407407407407407,
    0.89577397910731249,
    0.92592592592592593]


# Type 2: Classifier-Level Fusion

In [19]:
def get_type_2_model(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))

    ct_model = Conv2D(16, (2, 2), activation='relu')(ct_input)
    ct_model = Conv2D(36, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(64, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(144, (2, 2), activation='relu')(ct_model)
    ct_model = AveragePooling2D((23, 23))(ct_model)
    ct_model = Flatten()(ct_model)

    pet_model = Conv2D(16, (2, 2), activation='relu')(pet_input)
    pet_model = Conv2D(36, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(64, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(144, (2, 2), activation='relu')(pet_model)
    pet_model = AveragePooling2D((23, 23))(pet_model)
    pet_model = Flatten()(pet_model)

    x = concatenate([ct_model, pet_model])
    x = Dense(864, activation='relu')(x)
    x = Dense(288, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=[ct_input, pet_input], outputs=output)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()
    
    return model

In [20]:
f1s_2, accs_2 = train_n_sessions(get_type_2_model, 'type_II', 10, epochs=5, n_splits=1, val=False)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 40s - loss: 0.1151 - acc: 0.9597    
Epoch 2/5
41338/41338 [==============================] - 40s - loss: 0.0780 - acc: 0.9738    
Epoch 3/5
41338/41338 [==============================] - 40s - loss: 0.0630 - acc: 0.9795    
Epoch 4/5
41338/41338 [==============================] - 40s - loss: 0.0539 - acc: 0.9834    
Epoch 5/5
41338/41338 [==============================] - 40s - loss: 0.0460 - acc: 0.9860    
F1: 0.8724659994867847
Acc: 0.8820037986704653



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 40s - loss: 0.1165 - acc: 0.9593    
Epoch 2/5
41338/41338 [==============================] - 40s - loss: 0.0738 - acc: 0.9751    
Epoch 3/5
41338/41338 [===========

In [21]:
pp.pprint(f1s_2)
pp.pprint(accs_2)

[   0.87246599948678472,
    0.90167257982767357,
    0.93018777082330284,
    0.93512090016758442,
    0.94457603031738513,
    0.90103248552002013,
    0.94674835061262963,
    0.93365269461077849,
    0.93715651135005973,
    0.89035532994923861]
[   0.8820037986704653,
    0.90788224121557459,
    0.93114909781576449,
    0.93566001899335227,
    0.94444444444444442,
    0.90669515669515666,
    0.94634377967711303,
    0.93423551756885093,
    0.93755935422602088,
    0.89743589743589747]


# Type 3: Decision-Level Fusion

In [11]:
def get_type_3_model(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))

    ct_model = Conv2D(16, (2, 2), activation='relu')(ct_input)
    ct_model = Conv2D(36, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(64, (2, 2), activation='relu')(ct_model)
    ct_model = Conv2D(144, (2, 2), activation='relu')(ct_model)
    ct_model = AveragePooling2D((23, 23))(ct_model)
    ct_model = Flatten()(ct_model)
    ct_model = Dense(864, activation='relu')(ct_model)
    ct_model = Dense(288, activation='relu')(ct_model)
    ct_model = Dense(2, activation='softmax')(ct_model)

    pet_model = Conv2D(16, (2, 2), activation='relu')(pet_input)
    pet_model = Conv2D(36, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(64, (2, 2), activation='relu')(pet_model)
    pet_model = Conv2D(144, (2, 2), activation='relu')(pet_model)
    pet_model = AveragePooling2D((23, 23))(pet_model)
    pet_model = Flatten()(pet_model)
    pet_model = Dense(864, activation='relu')(pet_model)
    pet_model = Dense(288, activation='relu')(pet_model)
    pet_model = Dense(2, activation='softmax')(pet_model)

    predictions = average([ct_model, pet_model])

    model = Model(inputs=[ct_input, pet_input], outputs=predictions)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()
    
    return model

In [12]:
f1s_3, accs_3 = train_n_sessions(get_type_3_model, 'type_III', 10, epochs=5, n_splits=1, val=False)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 43s - loss: 0.1419 - acc: 0.9503    
Epoch 2/5
41338/41338 [==============================] - 43s - loss: 0.1124 - acc: 0.9666    - ETA: 2s - loss: 0.1 - ETA: 1s - loss:
Epoch 3/5
41338/41338 [==============================] - 42s - loss: 0.0927 - acc: 0.9709    
Epoch 4/5
41338/41338 [==============================] - 43s - loss: 0.0865 - acc: 0.9730    
Epoch 5/5
41338/41338 [==============================] - 42s - loss: 0.0808 - acc: 0.9750    
F1: 0.7877437325905292
Acc: 0.8190883190883191



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Epoch 1/5
41338/41338 [==============================] - 42s - loss: 0.1432 - acc: 0.9561    
Epoch 2/5
41338/41338 [==============================] - 42s - loss: 0.1079 - acc: 0.9687

In [13]:
pp.pprint(f1s_3)
pp.pprint(accs_3)

[   0.78774373259052921,
    0.81475498115239631,
    0.75114678899082565,
    0.83197475677097033,
    0.82795412109895972,
    0.7892827239743232,
    0.78171173682743245,
    0.80707482993197277,
    0.81334050564819793,
    0.8123143397245477]
[   0.81908831908831914,
    0.83665716999050332,
    0.79392212725546063,
    0.84829059829059827,
    0.84686609686609682,
    0.82075023741690412,
    0.8141025641025641,
    0.83167141500474839,
    0.83523266856600187,
    0.83499525166191835]


# Baseline: Single-Modality CNNs

In [14]:
def get_single_modality_model(summary=False):
    print('Build model...')

    K.clear_session()
    
    model = Sequential()
    model.add(Conv2D(16, (2, 2), activation='relu', input_shape=(PATCH_HEIGHT, PATCH_WIDTH, 1)))
    model.add(Conv2D(36, (2, 2), activation='relu'))
    model.add(Conv2D(64, (2, 2), activation='relu'))
    model.add(Conv2D(144, (2, 2), activation='relu'))
    model.add(AveragePooling2D((23, 23)))
    model.add(Flatten())
    model.add(Dense(864, activation='relu'))
    model.add(Dense(288, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()

    print('Model built.')
    
    return model

In [15]:
f1s_c, accs_c = train_n_sessions(get_single_modality_model, 'ct', 10, mode='ct', epochs=5, n_splits=1, val=False)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Build model...
Model built.
Epoch 1/5
41338/41338 [==============================] - 23s - loss: 0.2252 - acc: 0.9133    
Epoch 2/5
41338/41338 [==============================] - 23s - loss: 0.1484 - acc: 0.9469    
Epoch 3/5
41338/41338 [==============================] - 23s - loss: 0.1198 - acc: 0.9586    
Epoch 4/5
41338/41338 [==============================] - 23s - loss: 0.0950 - acc: 0.9678    
Epoch 5/5
41338/41338 [==============================] - 23s - loss: 0.0819 - acc: 0.9724    
F1: 0.46368715083798884
Acc: 0.6353276353276354



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Build model...
Model built.
Epoch 1/5
41338/41338 [==============================] - 23s - loss: 0.2155 - acc: 0.9197    
Epoch 2/5
41338/41338 [==============================] - 23s - loss: 0.

In [16]:
pp.pprint(f1s_c)
pp.pprint(accs_c)

[   0.46368715083798884,
    0.61586121437422547,
    0.45690259285213736,
    0.47830614370010416,
    0.55989750160153751,
    0.40750275836704669,
    0.57279084551811821,
    0.58119122257053291,
    0.64211807668898357,
    0.44877702942219072]
[   0.63532763532763536,
    0.70560303893637222,
    0.6320037986704653,
    0.64316239316239321,
    0.6737891737891738,
    0.61752136752136755,
    0.68091168091168086,
    0.68281101614434947,
    0.72079772079772075,
    0.63081671415004748]


In [22]:
f1s_p, accs_p = train_n_sessions(get_single_modality_model, 'pet', 10, mode='pet', epochs=5, n_splits=1, val=False)

Round 1 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Build model...
Model built.
Epoch 1/5
41338/41338 [==============================] - 23s - loss: 0.1173 - acc: 0.9592    
Epoch 2/5
41338/41338 [==============================] - 23s - loss: 0.0862 - acc: 0.9695    
Epoch 3/5
41338/41338 [==============================] - 23s - loss: 0.0740 - acc: 0.9748    
Epoch 4/5
41338/41338 [==============================] - 23s - loss: 0.0641 - acc: 0.9791    
Epoch 5/5
41338/41338 [==============================] - 23s - loss: 0.0577 - acc: 0.9815    
F1: 0.9276759884281581
Acc: 0.9287749287749287



Round 2 out of 10
-----------------------------------------------------------------------------------------------------
Train...
Build model...
Model built.
Epoch 1/5
41338/41338 [==============================] - 23s - loss: 0.1219 - acc: 0.9574    
Epoch 2/5
41338/41338 [==============================] - 23s - loss: 0.0

In [23]:
pp.pprint(f1s_p)
pp.pprint(accs_p)

[   0.92767598842815813,
    0.94128878281622907,
    0.92963855421686747,
    0.91464019851116629,
    0.92805055906660183,
    0.9092265605570754,
    0.86250000000000004,
    0.92247684056557777,
    0.89876293865185564,
    0.87921710018027299]
[   0.92877492877492873,
    0.94159544159544162,
    0.93067426400759734,
    0.91832858499525161,
    0.92972459639126304,
    0.91334283000949668,
    0.87464387464387461,
    0.92450142450142447,
    0.90479582146248816,
    0.88865147198480532]


In [ ]:
def get_two_path_cascade(summary=False):
    K.clear_session()

    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    
    conv1_local = Conv2D(64, (7, 7), activation='relu')(x)
    pool1_local = AveragePooling2D((23, 23))(conv1_local)
    
    conv2_local = Conv2D(64, (3, 3), activation='relu')(pool1_local)
    pool2_local = AveragePooling2D((23, 23))(conv2_local)

    conv1_global= Conv2D(160,(13,13), activation= 'relu')(x)
    
    combine = merge([pool2_local,conv1_global], mode= 'concat', concat_axis=1)
    conv1_combine = Conv2D(5, (21,21), activation='relu')(combine)
    output = Flatten()(conv1_combine)
    
    output = Dense(2, activation='softmax')(output)

    model = Model(inputs=[ct_input, pet_input], outputs=output)

    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    if summary:
        model.summary()
    return model


In [ ]:
c_matrices_two, accuracies_two = train_n_sessions(get_two_path_cascade, 'Casc-CNN-two', 10)

In [ ]:
def get_local_path_cascade(summary=False):
    K.clear_session()
    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    
    conv1_local = Conv2D(64, (7, 7), activation='relu')(x)
    pool1_local = AveragePooling2D((23, 23))(conv1_local)
    
    conv2_local = Conv2D(64, (3, 3), activation='relu')(pool1_local)
    pool2_local = AveragePooling2D((23,23))(conv2_local)
    
    conv1_combine = Conv2D(5, (21,21), activation= 'relu')(pool2_local)
    
    output = Flatten()(conv1_combine)
    output = Dense(2, activation= 'softmax')(output)
    
    model = Model(inputs=[ct_input, pet_input], outputs=output)
    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    if summary:
        model.summary()
        
    return model
    
    

In [ ]:
c_matrices_local, accuracies_local = train_n_sessions(get_local_path_cascade, 'Casc-CNN-local', 10)

In [ ]:
def get_global_path_cascade(summary=False):
    K.clear_session()
    ct_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    pet_input = Input(shape=(PATCH_HEIGHT, PATCH_WIDTH, 1))
    x = concatenate([ct_input, pet_input])
    x = Reshape((PATCH_HEIGHT, PATCH_WIDTH, 2, 1))(x)
    
    conv1_local = Conv2D(160, (13, 13), activation='relu')(x)
    conv1_combine = Conv2D(5, (21,21) ,activation= 'relu')(conv1_local)
    
    output = Flatten()(conv1_combine)
    output = Dense(2, activation= 'softmax')(output)
    
    model = Model(inputs=[ct_input, pet_input], outputs=output)
    model.compile(optimizer=Adam(lr=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    if summary:
        model.summary()
        
    return model

In [ ]:
c_matrices_global, accuracies_global = train_n_sessions(get_local_path_cascade, 'Casc-CNN-local', 10)